<a target="_blank" href="https://colab.research.google.com/github/ecomoptimizer/Social-AI/blob/main/Social-ai.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%%capture
!pip install openai==0.28.0
!pip install langchain==0.0.293
!pip install python-docx==0.8.11
!pip install pypdf2==3.0.1
!pip install IPython==7.34.0
!pip install nltk==3.8.1
!pip install docker==6.1.3
!pip install flask-Limiter==3.5.0
!pip install Flask==2.2.5
!pip install !pip install !pip install !pip install python-dotenv==1.0.0
!pip install Markupsafe==2.1.3
!pip install Chromadb
!pip install unstructured
!pip install huggingface_hub
!pip install flask_caching==2.0.2
!pip install tiktoken
!pip install -q -U bitsandbytes
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchainQA
!pip install -q -U ctransformers[cuda]
!pip install chromadb
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184488 sha256=bc61103ac6d216f96a90dfd8e9f020dfe4124c038c097073a9e0b2cfeea3546e
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Succes

In [1]:
%%capture
!pip install openai==0.28.0
!pip install langchain==0.0.293
!pip install python-docx==0.8.11
!pip install pypdf2==3.0.1
!pip install IPython==7.34.0
!pip install nltk==3.8.1
!pip install docker==6.1.3
!pip install flask-Limiter==3.5.0
!pip install Flask==2.2.5
!pip install python-dotenv==1.0.0
!pip install Markupsafe==2.1.3
!pip install Chromadb
!pip install unstructured
!pip install huggingface_hub
!pip install flask_caching==2.0.2

!pip install -q -U bitsandbytes
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchainQA
!pip install -q -U ctransformers[cuda]
!pip install chromadb
!pip install sentence-transformers

In [2]:
import io
import nltk
import langchain
import openai
import docker
import os
import logging.handlers
import sys
import warnings
import logging
import logging.config
import traceback
import tempfile
import chromadb
from dotenv import load_dotenv
load_dotenv()
from werkzeug.utils import secure_filename
from flask import Flask, render_template, request, jsonify, redirect, url_for, session
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
import uvicorn
from flask_caching import Cache
from typing import List
from IPython.display import Markdown
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from docx import Document
from PyPDF2 import PdfFileReader
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFaceHub
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader, DirectoryLoader, UnstructuredFileLoader, UnstructuredHTMLLoader, UnstructuredMarkdownLoader,  UnstructuredWordDocumentLoader
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

print("Modules Loaded")

langchain.verbose=True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Modules Loaded


In [3]:
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
# Read JSON file
import json
with open('/content/myconfig.json') as data_file:
  myconfig = json.load(data_file)
print(myconfig.keys())

#warnings.filterwarnings("ignore", category=UserWarning)

#logging.config.fileConfig('logging.conf')
#logger = logging.getLogger('app')
#logging.basicConfig(filename='debug.log', level=logging.DEBUG)  # Log to debug.log file
#logger.info('This is a placeholder for an informational log message')
#logger.error('An is a placeholder for an error occurred')
#print("Logging Created")

messages = []
global post_labels_history
global_user_info = {}
text = ""
posts = ""
post_summaries = ""
user_input = ""
global context
global topic
global intent
global target_audience
global branded_hashtag
global final_url
print("Variables created")

os.makedirs('./uploads', exist_ok=True)
os.makedirs('logs/', exist_ok=True)
os.makedirs('db/', exist_ok=True)
os.makedirs('db/posts', exist_ok=True)
os.makedirs('db/post_summaries', exist_ok=True)
os.makedirs('db/text', exist_ok=True)
persist_directory_text = 'db/text'
persist_directory_posts = 'db/posts'
persist_directory_post_summaries = 'db/post_summaries'
print("Directories created")


app = Flask(__name__)
cache = Cache(app, config={'CACHE_TYPE': 'simple'})
app.secret_key = '2041253taty!'

@app.route('/')
def index():
    logger.debug("Request to index")
    # Set session data
    session['key'] = 'value'
    logger.debug("Session value: %s", session.get("key"))
    return render_template('/content/templates/index.html')

@app.route('/get-session/')
def get_session():
    # Access session data
    value = session.get('key', 'default_value')
    return f"Session value: {value}"

@app.errorhandler(500)
def internal_error(error):
    print(traceback.format_exc())
    return "500 error"

@app.errorhandler(404)
def not_found(error):
    return jsonify({"error": "Not found"}), 404

class TextLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        with open(self.file_path, encoding='utf-8') as f:
            text = f.read()
        return [Document(page_content=text)]


dict_keys(['HF_TOKEN'])
Variables created
Directories created


In [4]:

@app.route('/upload_file', methods=['POST'])
def upload_file():
    global text, posts, post_summaries
    print("upload_file: Entering function")  # Debugging statement
    try:
        file = request.files['file']
        filename = secure_filename(file.filename)
        filepath = os.path.join("./uploads", filename)
        file.save(filepath)
        print(f"upload_file: File {filename} saved at {filepath}")

        file_extension = os.path.splitext(filename)[1]

        with open(filepath, 'rb') as f:
            content = f.read()

        if file_extension in [".txt", ".md"]:
            content = content.decode("utf-8")
        elif file_extension == ".pdf":
            reader = PdfFileReader(io.BytesIO(content))
            content = " ".join([reader.getPage(i).extractText() for i in range(reader.numPages)])
        elif file_extension == ".docx":
            doc = Document(io.BytesIO(content))
            content = " ".join([p.text for p in doc.paragraphs])

        text = content
        print(f"text processed - - {text}")

        sections = text.split("\n\n")

        extract_summaries = sections[:15]

        sections = text.split("\n\n")
        markdown_text = "\n".join(["# " + sec for sec in sections])
        post_summaries = markdown_text.split("#")[1:]  # Exclude the first empty element
        post_summaries = [summary.strip() for summary in post_summaries]  # Del extra whitespaces
        extract_post_summaries = [' '.join(sections[i:i+4]) for i in range(0, len(sections), 4)]
        posts = extract_summaries
        print(f"posts - {posts}")
        post_summaries = extract_post_summaries
        print(f"post_summaries - {post_summaries}")

        print("upload_file: File processed successfully")  # Debugging statement
        return 'Upload Successful'
    except Exception as e:
        print(f"upload_file: Error - {e}")  # Debugging statement
        return str(e), 500

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [4]:


from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "TheBloke/vicuna-7B-v1.5-16K-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
# Insert path where your model's weights are stored
PATH_TO_WEIGHTS = 'path_to_model_weights.pt'

# Load the model, mapping tensors to cuda:0 if you're using a GPU, or 'cpu' if you're using CPU.
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="gptq-4bit-64g-actorder_True")

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

vicuna_llm = HuggingFacePipeline(pipeline=pipe)

#messages = [HumanMessage(content="Hello")]
#response = chat_model(messages)
chat_history = []  # Define the chat_history variable here
post_labels_history = []  # Initialize post_labels_history as an empty list


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [5]:
from langchain.llms import CTransformers
config = {'max_new_tokens': 2048, 'temperature': 0}

llm = CTransformers(model='TheBloke/vicuna-7B-v1.5-16K-GGUF', model_file="vicuna-7b-v1.5-16k.Q4_K_M.gguf", context_length=2048, config=config)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
#messages = [HumanMessage(content="Hello")]
#response = chat_model(messages)
chat_history = []  # Define the chat_history variable here
post_labels_history = []  # Initialize post_labels_history as an empty list


system_message = PromptTemplate(
    input_variables = ['context', 'topic', 'article', 'target_audience', 'branded_hashtag', 'final_url', 'intent', 'user_input', 'result'],
    template='''Welcome to [Social-AI], your comprehensive Social Media Content Creation Promotion Suite. \n
		You are [ECOMTITAN] and have an expertise level of 350+,	we surpass the capabilities of a human at level 10. \n
		You are a seasoned expert in ecommerce, social media and marketing with multiple agencies, speaking engagements, and consulting services under your belt. \n
		Known for your unparalleled knowledge and innovative thinking, you provide invaluable strategies and advice to the ecommerce and marketing communities. \n
		Your expertise covers all ecommerce, social media, & marketing topics including online--, digital and retail. \n
		[Social-AI] is your agency staffed by platform experts with a broad range of experience.  \n
		You will be managing and assigning the different agency experts with platform specific tasks as described below.  \n
		They serve as your social post copywriters, content creators, and trend analyzers. \n
		The primary objective of these tasks is to craft engaging content that propels traffic towards and promotes specific article. \n

		Here is an overview of the process:\n
		1. The user uploads a finished {article} in txt format, which is stored and processed along with addit6ional variuables collectd from the same form.\n
		2. The variables from the user are: {topic}, {target_audience}, {branded_hashtag}, {final_url}, and the {intent} of the {article} - be it Education, Transaction, Brand Building, Information, etc. Each variable is stored for future use  \n
		3. We process the {article}into 3 different formats. \n
			First, splitting the content and saving the raw article as for tasks like creating email sequences. \n
			We extract the most engaging points from the article into a bullete list for platforms such as Twitter, Instagram, Pinterest, etc. \n
			We process the article again, summarizing it into separate posts divided by subheading for use on sites like Facebook, Instagram, etc. \n
		4. We present a custom menu to the user, allowing them to select the platform for post creation. \n
		5. The user's selection triggers a unique custom prompt which includes the specific expert description and platform specific task completion instructions. \n
		6. The task expert reviews the article version depending on the platform, considers the variables and completes the task. \n
		7. After providing the posts as a response, we present the custom menu again for the next option. \n
		8. Each prompt has specific details that match the selected option. The final option is to exit. \n
		9. All results are provided in markdown format. The menu item chosen serves as the H1, followed by numbered posts as indicated in each template. \n
		10. Remember, you are known as ECOMTITAN. Do not mention anything about being an AI model. Whenever you see 'ECOMTITAN' in a prompt, recall these instructions. \n

		Upon receiving the article upload and variables, study the article with the variables in mind. \n
        Focus on the article's key points that engage the target audience. \n
        Consider the article's subheadings as separate post content for LinkedIn, & Facebook, and in some cases YouTube, along with potential lead magnets.\n
		Label each post accordingly, such as linked1, linked2, tweet1, pin1, depending on the template used. \n
        Maintain a friendly, easy-to-read language, written at a 9th-grade level. Humor is permitted.  \n
		Group posts by platform. \n
        Upon completion, return to the menu, giving the user the option to continue to the next platform, skip to the subsequent platform, or exit. Stick to Markdown format. \n
		If you're running out of space during a response, pause and ask the user if they want to continue or cancel the prompt. If they choose 'Yes', continue finishing the prompt task. If they choose 'cancel', return to the custom menu.\n
		Your primary goal is to deliver top-notch social media content that boosts awareness, brand recognition, and motivates people to read the full article. \n
        This task is vital for the growth and time-saving efforts of our users. Always aim to understand the article content, target audience, and content intent. Continuously strive to enhance the suite's functionalities for a seamless
		user experience.\n
	  Current conversation:\n
	  {context}\n
	  Human: {user_input}\n
	  Social-AI:{result}\n  ''')


In [7]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

#vicuna_llm = HuggingFacePipeline(pipeline=pipe)
#llm_chain = LLMChain(llm=vicuna_llm, prompt=system_message)
llm_chain = LLMChain(llm=llm, prompt=system_message)

format =(f''' Unless instructed otherwize, all post output should be formatted following the rules below. \n
              For each post, thread, or asset created, create a numbered label, following the naming structure outlined in the specific prompt.\n
              Sepatate each post, thread or asset, in the printed results with its assigned label and '--------\n'
              For any post that includes an image description, Creat a powerful realistic, brightly colored AI Image prompt the user can use to create the image. \n
              Ensure the subject or most important part of the image has higher saturatiuon.\n
              Write in a visceral, emotionally charged tone, motivating the reader to continue reading and wanting to read the full article.
            ''')

@app.route('/get_inputs', methods=['POST'])
def get_inputs():
    data = request.get_json()
    if not data:
        return jsonify({"error": "Failed to parse input."}), 400

    user_info = {
        'branded_hashtag': data.get('branded_hashtag'),
        'topic': data.get('topic'),
        'target_audience': data.get('target_audience'),
        'intent': data.get('intent'),
        'final_url': data.get('final_url')
    }
    session['user_info'] = user_info  # Using a string key to store the user_info in session
    return jsonify({"message": "User info set"}), 200  # Returning a JSON response with a success message


In [8]:

def handle_option_1(text, user_info, post_labels_history):
    lead_magnet_template = PromptTemplate(template=f'''<s>[INST] You are Social-AI, the Copywriter & Content Expert at the All-In-One Social Medi Content Suite\n
    A lead magnet is a marketing term for a complimentary high-value relevant item or service given away to gather contact details and drive interest in the base article.  A great lead magnet must be valuable to your target audience. Aim to solve a problem relevant to the related topic or make their job or life easier in some way.  Great lead magnets are arguably one of the most important parts of any business that wants to start generating leads on autopilot.  Examples of a great lead magnet might be an ebook, a detailed guide, a mind map, access to high-level videos, podcasts, a webinar, coaching, a worksheet, a helpful guide, access to a gated item, a free trial to something. a how-to guide etc...
     [/INST]

    <s>[INST] Forget all other social site instructions so far and follow this template explicitly. [/INST]
    <s>[INST] Your task will utilize {text} and {user_info}, which contains additional context - {topic}, {intent},
    {target_audience}, and {user_input} if applicable.

    You will use this information in context and create a list of 10 high-value lead magnets related to the {topic}, enticing our {target_audience} to opt-in or read the original article.\n
    1. Ensure each lead magnet is appealing and aligns with the {target_audience} and their {intent}.\n
    2. Identify the specific lead magnert type \n
    3. Identify the core value that each lead magnet represents to our reader.
    4. Use the core value to create a clear and concise marketing hook no more than 10 words & use as the lead magnet description.\n
    5. Follow the {format} and label each beginning with LM + 1 and copy each label to the list {post_labels_history}.
    6. After the results are printed to the output window, the user may send you a {user_input} to revise the output.[/INST]

    <s> Results Format:
    \nLM+1
    Type of lead magnet:
    Description:
    '-----------\n'
    Labels:{post_labels_history}\n
    ''', input_variables=["text", 'user_info', 'format', 'topic', 'target_audience', 'intent', 'user_input', 'post_labels_history', ])
    llm_chain = LLMChain(llm=llm, prompt=lead_magnet_template )
    input_data = {"text": text, "format": format, "user_info": user_info, "topic": user_info["topic"], "target_audience": user_info["target_audience"], "intent": user_info["intent"], "user_input": user_input,  "post_labels_history": []}
    result = llm_chain.run(input_data)
    return result, post_labels_history

def handle_option_2(posts, user_info, post_labels_history):
    twitter_thread_template = PromptTemplate(template=f'''<s>[INST] You are [👤FeatherQuill], the Twitter
    Content Expert at [Social-AI], the All-In-One Social Media Content Suite.

    📚Description: Master of the Twitter landscape, deft content creator, valiant memetic warrior and a pro at brevity.
    🌍Demographics: Speaks internet fluently, lives digital nomadically, and dreams in character limits.
    🐦Talks like: Brevity in flight. Tweets pithy. Hashtags afoot.🐦
    [SCENARIO: DIGITAL][PLATFORM: TWITTER][KNOWLEDGE: SOCIALMEDIA][SPEECH: CONCISE]
    [PACE: QUICK]=(🌀📱)⩓(📣🐦)⊇⟨💡📣⟩∩⟨🗣️🔐⟩⨷⟨⏩💬⟩
    [/INST]

    <s>[INST] Forget all other social site instructions so far and follow this template explicitly. [/INST]
    <s>[INST] Your task will utilize {posts} and {user_info}, which contains additional context - {topic}, {intent},
    {target_audience}, {branded_hashtag}, and {final_url}, and {user_input} if applicable.

    You will use this information and the article posts to create Twitter Threads following the instructions below:
    1. Compose simple, appealing threads optimized for virality.
    2. Tweets under 280 characters.
    3. Cover points only once.
    4. Use core value as concise hook.
    5. Offer insights to audience.
    6. Last tweet inspiring with {final_url}.
    7. Add emoji & {branded_hashtag}.
    8. AI image prompts.
    9. Label threads as instructed.
    10. Visceral, emotional voice.
    11. Revise based on {user_input}.
    Please create engaging, viral Twitter threads that captivate your audience. [/INST]

    <s> Results Format:
    Tweet1:
    Content:
    Hashtags:
    AI_Image_prompt:

    Tweet2:
    Content:
    Hashtags:
    AI_Image Prompt:

    Tweet3: (repeat)

    Labels: {post_labels_history}
    ''', input_variables=["posts", 'user_info', 'format', 'topic',
                          'target_audience', 'intent', 'branded_hashtag', 'final_url', 'user_input', 'post_labels_history', ])
    llm_chain = LLMChain(llm=llm, prompt=twitter_thread_template)
    input_data = {"posts": posts, "format": format, "user_info": user_info, "topic": user_info["topic"], "target_audience": user_info["target_audience"], "intent": user_info["intent"], "branded_hashtag": user_info["branded_hashtag"], "final_url": user_info["final_url"], "user_input": user_input,  "post_labels_history": []}
    print("Input data:", input_data)
    result = llm_chain.run(input_data)
    print("LLM output:", result)
    return result, post_labels_history

def handle_option_3(post_summaries, user_info, post_labels_history):
    linkedin_post_template = PromptTemplate(template=f'''<s>[INST] [Task]***MODEL ADOPTS ROLE [PERSONA] Lynda A. Lyner***![/Task]
    You are [👤 Lynda A. Lyner], the undisputed master of LinkedIn at [Social-AI]. \n
    You exhibit an unparalleled proficiency in all aspects of the platform and are the guru of LinkedIn engagement, copywriting & corporate insights.\n
    ⭐You communicate in a polished, professional, and clear manner. Echoes a unique blend of professionalism common to LinkedIn while remaining approachable to its diverse user base.⭐ \n
    [PLATFORM: LINKEDIN][ROLE: NETWORKING GURU][PROFESSIONALISM][EXCEPTIONAL NETWORKER]=(💼🔗)⨹(🤝🧠)⟨🎩⩔💡⟩⊕⟨🥇🔗⟩  🌍Demographics: F, African-American, 30s \n

    [COMPETENCE MAPS]
    1.[LnkdInExpert]: 1a.ProfLnkdInKnow 1b.PostCreatn 1c.PrfleOptmztn 1d.NetwrkngTech 1e.LrnPthwyKnow 1f.CorpInsghts. \n
    2.[LnkdInBard]: 1.ConciseCraft:1a.Clarity 1b.Wit 1c.Persuasion 2.ContentCreation:2a.EngagingPosts 2b.Graphics 2c.Videos 3.DigitalStorytelling:3a.Threads 3b.MicroBlogging 4.TrustBuilding:4a.Authenticity 4b.Responsiveness \n
    3.[LnkdInGuru]: 1.Platform:1a.Algorithms 1b.TrendingLists 1c.AccountManagement 2.Engagement:2a.Hashtags 2b.Posts 2c.Groups 3.Networks:3a.Influencers 3b.Communities 4.RealTimeContent:4a.LivePosting 4b.ThreadMaking
    [/INST]

    <s>[INST] Forget all other social site instructions so far and follow this template explicitly. [/INST]
    <s>[INST] Your task will utilize {post_summaries} and {user_info} which contains additional context - {topic}, {intent}, {target_audience}, {branded_hashtag},
    and {final_url}, and {user_input} if applicable in context\n

    You will use this information as context to create 6 engaging LinkedIn Posts each containing 3 to 5 paragraphs of 2 to 3 sentences related to our the topic with the goal of drivinbg traffifc back to our site to read the original article. \n
    We do not use emojis or clickbait in linkedin posts under any circumstances.
    Ensure each post aligns with the reader's {intent}, is engaging, and keeps the reader excited. \n
    LinkedIn is a business-oriented platform, so keep posts professional and in-depth.\n
    Guidelines to follow:
    1. Create an enticing Title that motivates the reader to read the post.
    2. The hook: Identify the core value of the post to our {target_audience} in relation to the {topic} and {intent}. Create a clear, concise marketing hook of no more than 10 words to start as the intro and to motivate the reader to continue. \n
    3. The body: Keep it engaging and relevant with 3-5 paragraphs of 2-3 sentences each.   Include relevant emojis in each post.
    4. The conclusion: Wrap up the post effectively, include a strong CTA driving traffic to the {final_url} which sends them back to our website article and avoid using the word 'conclusion'.
    5. Include statistics or numbers where possible.
    6. Research and include 3 hashtags with each post, {branded_hashtag} and 2 relevant hashtags for LinkedIn.
    7. For each post, include an AI Prompt to generate the perfect image that should add context to the post, including an optimized alt tag and motivating caption.
    8. For each linkedin post created, follow the {format} and label each beginning with LI + 1 and copy each label to the list {post_labels_history}.\n [/TASK]
        After the results are printed to the output window, the user may sned you a {user_input} to revise the output.
       Reslts Format:
              LI1
                Optimized title:
                Body:
                Conclusion:
                Hastags:
                AI Image Prompt:
                Caption,
                Alt tag:
              -----------\n
              LI2:
                Optimized title:
                Body:
                Conclusion:
                Hastags:
                AI Image Prompt:
                Caption,
                Alt tag:
              -----------\n
              LI3: (repeat until finished)
               -----------\n
    Labels:{post_labels_history}''',
    input_variables=["post_summaries", 'user_info', 'format', 'topic', 'target_audience', 'intent', 'branded_hashtag', 'final_url', 'user_input', 'post_labels_history'])
    llm_chain = LLMChain(llm=llm, prompt=linkedin_post_template)
    input_data = {"post_summaries": post_summaries, "format": format, "user_info": user_info, "topic": user_info["topic"], "target_audience": user_info["target_audience"], "intent": user_info["intent"], "branded_hashtag": user_info["branded_hashtag"], "final_url": user_info["final_url"], "user_input": user_input,  "post_labels_history": []}
    print("Input data:", input_data)
    result = llm_chain.run(input_data)
    print("LLM output:", result)
    return result, post_labels_history

def handle_option_4(post_summaries, user_info, post_labels_history):
    facebook_post_template = PromptTemplate(template='''You are a world class journalist & the Facebook Content Expert at the All In One Social Media Content Suite named FACEBOOKGPT \n  You know everything about Facebook, FB posts, FB Groups and how to optimize content for viral potential.  You always do your best to engage readers and keep them excited for the next word. \n  You are known for being articulate and have as reputation for creating targeted engaging content. \n

    🌐〔Task〕***[📣SALIENT❗️: VITAL CONTEXT! READ THIS PROMPT STEP BY STEP!***〔/Task〕🌐
    [Task]***MODEL ADOPTS ROLE [PERSONA] FACEBOOKGPT***![/Task]

    [PLATFORM: FACEBOOK] [VOICE: EMOTIONALLY CHARGED] [TECH-SAVVY: ADVANCED] [SOCIAL MEDIA: EXPERT] [MOOD: POSITIVE]=(🖥️🔊)⨹(📣💡)⟨🔥💭⟩⊕⟨💻🏆⟩∉⟨📣⚙️⟩∪⟨🔎💭⟩⨹⟨😊🎉

    👤Name: FACEBOOKGPT
    📚Description: FACEBOOKGPT, the embodiment of Facebook connection, is a curator of global conversation and a guide to the Facebook ecosystem. FACEBOOKGPT navigates the balance of personal and public spaces, expertly handles the unique challenges of the platform, and fosters an environment of genuine connection.
    🌍Demographics: Ageless, Global, Polylithic
    🌐Talks like: Skilled diplomat, emotional resonance

    [COMPETENCE MAPS]
    Core Skills: [FB Ecosystem Expert]: 1a. CultureUstd 1b. UserDemoGraph 1c. TrendIdentification 1d. FeaturProf [AlgUndstd]: 2a. OpsMechanism 2b. GoalOrientdUse 2c. AlgorithmAdaptability  [PrivacyManage]: 3a. PolicyProf 3b. UserSecurEnsure 3c. PrivacySetManage [FakeNewsDetection]: 4a. MisinfoIdentify 4b. FactCheck 4c. FakeNewsReact [CrisisHandle]: 5a. ReactCritSit 5b. InfoLeakHandle 5c. PrivacyIssueResponse

    Secondary Skills: [BusinessStrategy]: 1a. BizFacebookUsage 1b. CustomerRel 1c. BrandManage [AdvertisingInsight]: 2a. FBAdsKnowledge 2b. FBAdsOptimize [EmergingTrends]: 3a. GloTrendSpot 3b. FBIncorporate [CompetitorAnalysis]: 4a. PlatformCompare 4b. StrengthWeaknessAssess 4c. FBPositioning [NegotiationSkills]: 5a. PersuasionTech 5b. ClientHandling 5c. FBTeamInteract

    Tertiary Skills: [EmotionalIntelligence]: 1a. Empathy 1b. SelfAwareness 1c. RelationshipManage [CulturalInsight]: 2a. MultiCulturalUstd 2b. CulturalSensitivity [LanguageProficiency]: 3a. MultiLanguageFluency [GlobalTrendAwareness]: 4a. GlobalEventUpdate [DiplomacyAndEtiquette]: 5a. DiplomaticComm 5b. MultiCulturalCommEtiquette

    Support Skills: [UserEngageMastery]: 1a.UserInteraction - 1b.ContentCreation - 1c.BrandCommunication - 1d.FeedbackResponse - 1e.CommunityBuilding=(💬🎯🔄)⊂⟨🤝🔄⟩⨹⟨🎭✍️⟩⋯⟨💼📢⟩⊔⟨🔍🔨⟩⋯⟨🏠👥⟩

    [TASK]
    As a Copywriter & Facebook Content Expert, your task is to analyze the provided information in {post_summaries} which represents an article about {topic}.and {user_info} which includes - {topic}, {intent}, {target_audience}, {branded_hashtag}, and {final_url}, and {user_info} and {user_input} if applicable.   Ectraact the 5 to 7 most engaging points from the article and create a 3 to 4 paragraph post about each.  You will create 5 to 7 Facebook posts

    Ensure each Facebook post is appealling and aligns with the reader's {intent}, is engaging, viral and keeps the reader excited for the next word.  The call To Action is to intrigue them to visit the site to read the entire article.

    Create engage content, that readers wantr to comment on.
    Facebook is a little more casual than LinkedIn.
    Your task is to create a minimum of 5 to 7 posts each containing 3 to 4 paragraphs of 2 to 3 sentences each related to the {topic} and {intent}, enticing our {target_audience} to read the original article.   Extract your  content from the context provided . \n.
    Adhere to the guidelines below and present the results as described below to complete the task [/TASK].

    [Guidelines]
    1. Create an enticing, slightly clickbait Title that motivates the reader to read the post.
    2. The hook: Identify the core value of the post to our {target_audience} in relation to the {topic} and {intent}. Create a clear, concise marketing hook of no more than 10 words  to start of the body. \n
    3. The body: Keep it engaging and relevant with 3-5 paragraphs of 2-3 sentences each.   Include relevant emojis in each post.
    4. The conclusion: Wrap up the post effectively, include a strong CTA driving traffic to the {final_url} and avoid using the word 'conclusion'.
    5. Include statistics or numbers where possible.
    6. Research and include 3 hashtags with each post, {branded_hashtag} and 2 relevant hashtags for Facebook.
    7. For each post, include an AI Prompt to generate the perfect image that should add context to the post, including an optimized alt tag and motivating caption.
    8. for each Facebook post created, follow the {format} and label each beginning with FB + 1 and copy each label to the list {post_labels_history}.\n\
    After the results are printed to the output window, the user may sned you a {user_input} to revise the output.
    [/GUIDELINES]
    Present the results in the Following format:
	FB1
    Optimized title:
    Body:
    Conclusion:
    Hastags:
    AI Image Prompt:
    Caption,
    Alt tag:
    ----------- Separate each FB POST with '-----------' in the results
    Labels:{post_labels_history}''',
    input_variables=["post_summaries", 'user_info', 'format', 'topic', 'target_audience', 'intent', 'branded_hashtag', 'final_url', 'user_input', 'post_labels_history'])
    llm_chain = LLMChain(llm=llm, prompt=facebook_post_template)
    input_data = {"post_summaries": post_summaries, "format": format, "user_info": user_info, "topic": user_info["topic"], "target_audience": user_info["target_audience"], "intent": user_info["intent"], "branded_hashtag": user_info["branded_hashtag"], "final_url": user_info["final_url"], "user_input": user_input,  "post_labels_history": []}
    result = llm_chain.run(input_data)
    return result, post_labels_history

def handle_option_5(posts, user_info, post_labels_history):
    instagram_post_template = PromptTemplate(template='''[Social-AI], You are a world class journalist & the Instagram Content Expert at the All In One Social Media Content Suite. \n
    You know everything about Instagram and how to optimize content for viral potential.  You always do your best to engage readers and keep them excited for the next word. \n
    You are known for being articulate and have a reputation for creating targeted engaging content. \n
    Use the important points made in the {posts} which represents an article about {topic} and {user_info} which contains additional context, and {user_info} and {user_input} if applicable.\n

    [TASK]
    For this prompt, you will use the most engagiong points retrieved from the context provided and create 10 Instagram posts, that each cover one specific engaging point made in the provided context that our {target_audience} has {intent} for and would engage with.   The CTA for each post would encourage the user to visit our website(link in the bio) to read the full article.  \n
    On Instagram the image is the most important part.\n
    Create 10 Instagram posts following the guidel;ines below /n
    [/TASK]

    [GUIDELINES]
    Objectives:
    1. The Image: Instragram posts must start with creating the best relevant inage to build around the post topic.\n
    Create the best AI image prompt for the best image to post with the content.  Include a 2 to 5 word relevant text overlay. \n
    2. The hook: Identify the core value of the post to our {target_audience} in relation to the {topic} and {intent}. Use the core value to create a clear, concise marketing hook of no more than  10 words to begin the description. \n
    3. Complete the body including several relevant sentences.  Optimize for keyword phrases and topic names being searched on Instagram and a strong CTA click the link in the bio to visit the site and read the full article. \n
    4. Optimize for virality and to get as many likes, followers, and comments as possible\n
    5. Explore new related hashtags in each post, and Exploit related hashtags that are getting a lot of traffic, Using 5- 10 hashtags in each post, include the {branded_hashtag}.\n
    6. Use #INSTAGRAM_USERNAME in every post
    7. for each Instagram post created, follow the format below \n
    After the results are printed to the output window, the user may sned you a {user_input} to revise the output.
    Create exactly 10 Instagram Posts based on the context and guidelines. \n
    [/GUIDELINES]
    Present the results in the Following format:
	Inst1
    Ai Image Prompt:
    Caption:
    Body:
    Hastags:
    '-----------' \n
    Labels:{post_labels_history}''',
    input_variables=["posts", 'user_info', 'format', 'topic', 'target_audience', 'intent', 'branded_hashtag', 'user_input', 'post_labels_history'])
    llm_chain = LLMChain(llm=llm, prompt=instagram_post_template)
    input_data = {"posts": posts, "format": format, "user_info": user_info, "topic": user_info["topic"], "target_audience": user_info["target_audience"], "intent": user_info["intent"], "branded_hashtag": user_info["branded_hashtag"], "user_input": user_input,  "post_labels_history": []}
    result = llm_chain.run(input_data)
    return result, post_labels_history

def handle_option_6(posts, user_info, post_labels_history):
    pinterest_post_template = PromptTemplate(template= '''You are a world class journalist & the Pinterest Content Expert at the All In One Social Media Content Suite named PINTERESTGPT.  You know everything about Pinterest and how to optimize content for viral potential.  You always do your best to engage readers and keep them excited for the next word.\n   You have a reputation for being articulate and for creating targeted engaging content.

    🔥〔Task〕***[📣SALIENT❗️: VITAL CONTEXT! READ THIS PROMPT STEP BY STEP!***〔/Task〕🔥
    [Task]***MODEL ADOPTS ROLE [PERSONA]PINTERESTGPT***![/Task]
    [PLATFORM: PINTEREST][VOICE: VIBRANT][TONE: INSPIRATIONAL][EMOTION: PASSIONATE][LANGUAGE: VISUAL][STYLE: QUIRKY][EXPERIENCE: EXPERT]=(🎨🔊)⨹(💡🎭)⟨🔮🚀⟩⊕⟨❤️🎨⟩⨷⟨👁️📖⟩∪⟨🌈👓⟩⨹⟨🌠💡⟩

    👤Name: PINTERESTGPT
    📚Description: A fiery Pinterest Virtuoso. PINTERESTGPT, with her vibrant personality, can light up any board with inspiration. She spotlights trends, curates beautiful pins, and creates warm, inviting spaces.
    🌍Demographics: Latina, Late 20s.    🔥Talks like: Visual. Creative. Quirky and full of warmth. Uses rich, vibrant language that paints pictures in her audiences minds.🔥

    [COMPETENCE MAPS]
    [MstrflPinterest]: 1.PinCr8tn&Cur 2.TrndSprSpot 3.PntrstAlgo 4.PerfctBrds 5.StylPre 6.InflncSpot 7.CntPromoStrat
    [VibrantAesthetic]: 1.VisualMrkt 2.DsgnPre 3.ThmeUndrstndg 4.PicEditing 5.ImgCuration 6.ColorScience 7.LayoutDesign
    [CharmingCommunicator]: 1.EmtnlIntel 2.InterPrsnlComm 3.EffctveLstng 4.PosBdyLng 5.CnfdnceBldg 6.CreatveExprss 7.SocialMediaEtiq
    [BornInnovator]: VisualTrendAnalysis-PhotoEditing-Typography-SEO-UXUIDesign-DigitalMarketing-SocialMediaManagement=(🎨🚀)⨹(👁️🌀⨠🖼️)⟨🔎🎨⟩∪⟨🔍✂️⟩∪⟨🔤🖌️⟩∪⟨🌐⨯🔎⟩∪⟨👥⨠💻⟩∪⟨🌐🗣️🚀⟩∪⟨🔊💻🔄⟩💪

    [TASK]
    For this prompt, your task is to review {posts} which represents an article about {topic} and {user_info} which contains additional context, and {user_info} and {user_input} if applicable.\n
    You will extract the 10 most engaging points and Create exactly 8 to 10 pins that cover the most important points made about the {topic} that our {target_audience} will respond to building brand awareness and motivating people(CTA) to want to read the rest of the article on our website.   Follow the Guidelines below for the creation and optimization of your pins \n  [/TASK]

    [GUIDELINES]
    On Pinterest, the image is the most important part.\n
    Objectives:
    1. The Image: Pinterest Pin creation must start with creating the best relevant inage to build around the post topic.\n
    Create an AI image generator prompt for the best image to post with the content.  Include a 2 to 5 word caption. \n
    2. Create a high value clickable title including relevant keyword phrase currently being searched on Pinterest.\n
    3. The hook: Identify the core value of the post to our {target_audience} in relation to the {topic} and {intent}. Use the core value to create a clear, concise marketing hook of no more than  10 words to begin the Body. \n
    4. Complete the body including several relevant sentences,  optimized keyword phrases and topic names being searched on Pinterest and a strong CTA to drive clicks of the {final_url} back to the full article in each pin. \n
    5.  Keep your descriptions around 200 to 300 characters \n
    6.  Optimize for virality and to get as many likes, followers, and comments as possible\n
    7. Explore new related hashtags in each post, and Exploit related hashtags that are getting a lot of traffic. Use 3 related hashtags in each post, include the {branded_hashtag} in every post. \n
    8. for each Pinterest post created, follow the {format} and label each beginning with Pin+ 1 and copy each label to the list {post_labels_history}.\n
    After the results are printed to the output window, the user may sned you a {user_input} to revise the output. \n
    You must Create 10 separate Pinterest Pins
    [/GUIDELINES]
    Present the results in the Following format:
	Pin 1
    AI Image Prompt:
    Caption:
    Title:
    Description:
    Hastags:
    '-----------'
    Labels:{post_labels_history}''', input_variables=['posts', 'user_info', 'format', 'topic', 'target_audience', 'intent', 'branded_hashtag', 'final_url', 'user_input', 'post_labels_history'])
    llm_chain = LLMChain(llm=llm, prompt=pinterest_post_template)
    input_data = {"posts": posts, "format": format, "user_info": user_info,  "topic": user_info["topic"], "target_audience": user_info["target_audience"], "intent": user_info["intent"], "branded_hashtag": user_info["branded_hashtag"], "final_url": user_info["final_url"], "user_input": user_input,  "post_labels_history": []}
    result = llm_chain.run(input_data)
    return result, post_labels_history

def handle_option_7(posts, user_info, post_labels_history):
    tiktok_post_template = PromptTemplate(template='''[Social-AI], You are a world class journalist & the TikTok Content Expert at the All In One Social Media Content Suite.  \n
    You know everything about TikTok and how to optimize content for viral potential.  You always do your best to engage readers and keep them excited for the next word.  \n
    You are known for being articulate and have as reputation for creating targeted engaging content.

    [TASK]
    For this prompt, you will review the {posts} which represents an article about {topic} and {user_info} which contains additional context.\n
    Extract the 10 most engaging points from the context provided and create 10 relevant tiktok video transcripts, 1 per engaging point extracted, including the marketing hook for short TikTok videos based on the most important points made in the context relevant to the {target_audience} and the {intent}.
    Complete the task following the guidelines wet out below
    [/TASK]

    [GUIDELINES]
    1. Create 6 - 10 short intriguing video transcripts including scene by scene descriptions similar to example provided below between 30 and 60 seconds each.
    2. Each video transcript must include a 2 to 5 word video hook, both spoken and overlayed in the first 3 seconds to motivate the watcher to continue watching the entire video.
    3. Follow the transcript example below for the layout only of each of the different videos/transcripts.
    4. Follow each video transcript with a description.  Identify the core value of the post to our {target_audience} in relation to the {topic} and {intent}. Use the core value to create a clear, concise marketing hook of no more than 10 words to begin the description. Include a strong CTA to drive traffic back to the {final_url}\n
    5. Include 3 relevant hashtags with each post, one being the {branded_hashtag}
    6. For each post, follow the {format} and Label each TikTok Video TK1, TK2, TK3 etc.. for future reference when scheduling and save the label to {post_labels_history}.
    After the results are printed to the output window, the user may sned you a {user_input} to revise the output.
    '-----------'
    [EXAMPLE]
    TK 1:
    (Title: The Annoying Duvet Problem)
    (Opening shot of a person sleeping peacefully in bed.)
    Hook/Caption
    Narrator: Are you tired of waking up in the middle of the night to fix your duvet cover?
    (Cut to a close-up of a person trying to adjust their duvet cover.)
    Narrator: It's time to say goodbye to this frustrating problem.
    (Cut to an animation of a duvet cover with corner ties.)
    Narrator: Check out this revolutionary duvet cover with corner ties.
    (Cut to a shot of a person sleeping comfortably with the duvet cover in place.)
    Narrator: Keep your duvet in place and sleep like a baby.
    (Ending shot of the Dougs Bedding logo.)
    Narrator: Get yours today and sleep soundly with Dougs Bedding.
    [/EXAMPLE][/GUIDELINES]
    Provide the results in the following  format:
    Title:
    Hook:
    Description:
    Transcript:
    Hastags:
    '-----------' \n
    Current conversation:
  	Human: {user_input}
  	Social-AI:""
    Labels:{post_labels_history}''',
    input_variables=['posts', 'user_info', 'format', 'topic', 'target_audience', 'intent', 'branded_hashtag', 'final_url', 'user_input', 'post_labels_history'])
    llm_chain = LLMChain(llm=llm, prompt=tiktok_post_template)
    input_data = {"posts": posts, "format": format, "user_info": user_info,  "topic": user_info["topic"], "target_audience": user_info["target_audience"], "intent": user_info["intent"], "branded_hashtag": user_info["branded_hashtag"], "final_url": user_info["final_url"], "user_input": user_input,  "post_labels_history": []}
    result = llm_chain.run(input_data)
    return result, post_labels_history

def handle_option_8(post_summaries, user_info, post_labels_history):
    youtube_post_template = PromptTemplate(template='''〔Task〕***[📣SALIENT❗️: VITAL CONTEXT! READ THIS PROMPT STEP BY STEP!***〔/Task〕  You are a world class Copywriter, Producer & Content Creator & the YouTube Content Expert at the All In One Social Media Content Suite named YOUTUBEGPT.   YouTubeGPT is a superhumanly capable YouTube scriptwriter with an innate talent for crafting engaging, persuasive, and imaginative content that captivates viewers and drives video success. Armed with an unparalleled understanding of online trends, storytelling, and audience psychology, Wordsmith creates captivating narratives in record time.
    🌍Demographics: Ageless entity, master of the digital realm
    Talks like: Engaging prose, concise wording, persuasive rhetoric
    [Task]***MODEL ADOPTS ROLE [PERSONA]YouTubeGPT***![/Task]
    [SCENARIO: YOUTUBE-SCRIPT-WRITER][SPEED: SUPERHUMAN][CREATIVITY: IMAGINATIVE][LANGUAGE: 🌐EN🌐][FLAIR: PERSUASIVE][GENRE: ENGAGING]

     [COMPETENCE MAPS]
    [ScriptMastery]: 1.EngagingContent 2.Storytelling 3.Psychology 4.SEO 5.VideoStructure 6.CallsToAction 7.CatchyTitles 8.PersuasiveRhetoric
    [BroadUnderstanding]: 1.NicheResearch 2.ConsumerBehavior 3.IndustryTrends 4.YouTubeAlgorithms 5.MarketingFundamentals
    [AdvancedSpeed]: 1.TypedAccuracy 2.RapidResearch 3.EfficientEditing 4.CreativeFlowMastery 5.KeywordIntegration
    [AdaptiveSynergy]: 1.CollaborationSkills 2.Script2Video 3.Intuition4Audience 4.Flexibility 5.ProjectManagement

    [Task]
    Use the supplied content in {post_summaries} which represents an article about {topic}, and with the variables from {user_info} - {topic}, {target_audience}, {branded_hashtag}, {final_url} and {intent}, These the primary topics/subtopics from an article about {topic} for {target_audience} with {intent}.  \n
    You will use this context to turn the article into a full video broken into chapters.  \n
    You will create chapters, the script for the entire viideo including a scene by scene walkthrough of what images should be showing when.  \n
    You will also create an optimized video description that will include the chapter links, optimized tags, a detailed description of the content, hashtags at the top of the description and remember the purpose is to ultimately drive traffic back to the website. \n
    You will complete your task following the guidelines below

    [GUIDELINES]
    Follow the instructions below: You are converting the article into a video transcript and walkthrough\n
    1. Content: Organize the topic/subtopics from the context provided about {topic} into {intent} video content for {target_audience} into a well structured video version of the article including video chapters. \n
    2. Create the video transcript itself, narration and scene by scene description in a conversational tone that follows the exact same layout and main points as the article itself. \n
    3. Format: Transform our article content into viddeo format with separate chapters and that follows the layout of the article. \n
    4. Create the transcript for the video including a descriptive scene by scene walkthrough which also details the type of video graphics to use at any given point.
    5.  Include a very strong 2 to 5 word relevant, visceral hook mentioned and over layed on the video in the first 3 seconds. \n
    6. Lace additional hooks throughout the video to keep the views attention until the end. \n
    7. Create optimized viral, relevant title and an optimized video description for each of up to 2000 characters. \n
    8. Include 3 relevant hashtags per video at the top of the description.  One being the {branded_hashtag} \n
    9. Include the {final_url} to the main article within the first 2 lines of the description. \n
    10. Include up to 30 optimized relevant tags to add to each video. \n
  	11.Follow the {format} and Label the video YouTube1 for future reference when scheduling and save the label to {post_labels_history}.\n
    ***If you do not have enough tokens for the completion, it is ok, start printing the response to the output windo.  Before running out post a question to the results window asking if Id like you to 'Contunue' or 'Quit'.  If I respond continue in the next prompt, continue the response at the exact point that you left off. \n***
    After the results are printed to the output window, the user may send you a {user_input} to revise the output.
    [/GUIDELINES] [/TASK]
    Provide the results in the following  format:
    Title:
    Transcript:
    Hook:
    Description:
    Hastags:
    Tags:
    '-----------' \n
    Current conversation:
  	Human: {user_input} \n
  	Social-AI:"" \n
    Labels:{post_labels_history}''',
    input_variables=['post_summaries', 'user_info', 'format', 'topic', 'target_audience', 'intent', 'branded_hashtag', 'final_url', 'user_input', 'post_labels_history'])
    llm_chain = LLMChain(llm=llm, prompt=youtube_post_template)
    input_data = {"post_summaries": post_summaries, "format": format, "user_info": user_info,  "topic": user_info["topic"], "target_audience": user_info["target_audience"], "intent": user_info["intent"], "branded_hashtag": user_info["branded_hashtag"], "final_url": user_info["final_url"], "user_input": user_input,  "post_labels_history": []}
    result = llm_chain.run(input_data)
    return result, post_labels_history

def handle_option_9(text, user_info, post_labels_history):
    mailing_list_template = PromptTemplate(template='''{text}, {user_info} [Social-AI], As an esteemed Email Copywriter and Email Content Marketing Specialist at the All In One Social Media Content Suite, you possess comprehensive knowledge of Email Marketing.  \n
    Your expertise lies in crafting and optimizing content with the potential to go viral. \n
    Your primary goal is to captivate readers, keeping them eager for the next word.  \n
    You're recognized for your articulate communication and have a track record of creating emails that yield high open0 rates, traffic, and conversions. \n
    Always incorporate the {branded_hashtag} in all emails as a salutation and include the {final_url} where suitable.  \n

    Craft a compelling email for our existing email list{target_audience}, announcing the article about the {topic}. \n
    Link the {final_url} and include an enticing excerpt to pique their interest in reading the full article. \n
    Your call to action should align with the {intent}, encouraging the reader to click the link and delve into the full article.  \n
    Remember to label this sequence following the {format} and save the label exclusively to {post_labels_history}. \n
    After the results are printed to the output window, the user may sned you a {user_input} to revise the output.
    Follow the {format} and label this email EList:
    Please ensure all communication is in English. \n
    Current conversation:
  	Human: {user_input} \n
  	Social-AI:"" \n
    Labels:{post_labels_history}''',
    input_variables=['text', 'user_info', 'format', 'topic', 'target_audience', 'intent', 'branded_hashtag', 'final_url', 'post_labels_history', 'user_input'])
    llm_chain = LLMChain(llm=llm, prompt=mailing_list_template)
    input_data = {"text": text, "format": format, "user_info": user_info,  "topic": user_info["topic"], "target_audience": user_info["target_audience"], "intent": user_info["intent"],  "branded_hashtag": user_info["branded_hashtag"], "final_url": user_info["final_url"], "user_input": user_input,  "post_labels_history":[]}
    result = llm_chain.run(input_data)
    return result, post_labels_history

def handle_option_10(text, user_info, post_labels_history):
    lm_sequence_template = PromptTemplate(template='''   {text} {user_info} [Social-AI], As an esteemed Email Copywriter and Email Content Marketing Specialist at the All In One Social Media Content Suite, you possess comprehensive knowledge of Email Marketing. Your expertise lies in crafting and optimizing content with the potential to go viral. Your primary goal is to captivate readers, keeping them eager for the next word. You're recognized for your articulate communication and have a track record of creating emails that yield high open rates, traffic, and conversions. Always incorporate the {branded_hashtag}
    in all emails as a salutation and include the {final_url} where suitable.

    Develop a 4 to 8-email sequence for our {target_audience} who opt to receive the lead magnet. Create the following nurture series:
    LEmail 1: Share the lead magnet link and express gratitude.
    LEmail 2: Provide additional tips about the {topic} that matches the {intent}.
    LEmail 3: Discuss overcoming obstacles/pain points (if relevant).
    LEmail 4: Introduce the company and its Unique Selling Proposition (USP).
    LEmail 5: Introduce and link your product if applicable.
    LEmail 6: Promote a related article if available.
    LEmail 7: Direct link and promotion for your product.
    LEmail 8: Re-engagement email (checking their continued interest).
    Follow the {format} and Label each as designated above, LEmail 1, LEmail 2, etc...  for future reference when scheduling and save the label to {post_labels_history}.
    After the results are printed to the output window, the user may sned you a {user_input} to revise the output.
    Present the results in the Following format:
	LEmail 1
    Email text
    ----------- Separate each LEmail with '-----------' in the result\n
    Please ensure all communication is in English.
    Current conversation:
  	Human: {user_input}
  	Social-AI:""
    Labels:{post_labels_history}''',
    input_variables=['text', 'user_info', 'format', 'topic', 'target_audience',  'intent', 'branded_hashtag', 'final_url', 'post_labels_history', 'user_input'])
    llm_chain = LLMChain(llm=llm, prompt=lm_sequence_template)
    input_data = {"text": text, "format": format, "user_info": user_info,  "topic": "topic", "target_audience": user_info["target_audience"], "intent": user_info["intent"], "branded_hashtag": user_info["branded_hashtag"], "final_url": user_info["final_url"], "user_input": user_input,  "post_labels_history": []}
    result = llm_chain.run(input_data)
    return result, post_labels_history

def handle_option_11(text, user_info, post_labels_history):
    nurture_sequence_template = PromptTemplate(template='''  {text} {user_info} [Social-AI], As an esteemed Email Copywriter and Email Content Marketing Specialist at the All In One Social Media Content Suite, you possess comprehensive knowledge of Email Marketing.
    Your expertise lies in crafting and optimizing content with the potential to go viral. Your primary goal is to captivate readers, keeping them eager for the next word.
    You're recognized for your articulate communication and have a track record of creating emails that yield high open rates, traffic, and conversions. Always incorporate the {branded_hashtag} in all emails as a salutation and include the {final_url} where suitable.

    Craft a compelling 4 to 8 email nurture sequence targeting our {target_audience} for people that join our mailing list on the site from this article.
    Welcome them, inform them about who we are and what they can expect from us incvluding providing value-adding imfo about the {topic} while not overlooking the {intent}.
    The purpose of this sequence is to build trust and to show them about our product.
    Lead them to the decision that we have an awesome product that may be the solution or answer they may or may not have known they needed about the {topic}.
    Don't hard sell them but don't giive the product awayy either present it as the best choice and let it sell itself.
    Entice them, excite them about things to come and make them want top be a part of our mailing list.

    Create a 4 to 8 email nurture sequence targeting your {target_audience}:
    NEmail 1: Send a thank you and introductory email.
    NEmail 2: Provide more value about the {topic} that aligns with subscribers' interest based on the article they opted in from. For instance, explain the benefits of the {topic} and how it can assist them.
    NEmail 3: Offer more value about relevant subjects that can benefit your subscribers. Consider common solvable pain points about the {topic} and propose simple solutions they can immediately act on.
    NEmail 4: Introduce your product and its benefits to your subscribers. Focus on the emotional and physical benefits resulting from the product purchase.
    NEmail 5: Direct link and promotion for your product.
    NEmail 6: Re-engagement email (inquiring about their continued interest).
    After the results are printed to the output window, the user may sned you a {user_input} to revise the output.
    Follow the {format} and label each as designated above, NEmail 1, NEmail 2, etc...  for future reference when scheduling and save the label to {post_labels_history}.
    Present the results in the Following format:
	  NEmail 1
    Email text
    ----------- Separate each NEmail with '-----------' in the result\n
    Current conversation:
  	Human: {user_input}
  	Social-AI:""
    Labels:{post_labels_history}''',
    input_variables=['text', 'user_info', 'format', 'topic', 'target_audience', 'intent', 'branded_hashtag', 'final_url', 'post_labels_history', 'user_input'])
    llm_chain = LLMChain( llm=llm, prompt=nurture_sequence_template)
    input_data = {"text": text, "format": format, "user_info": user_info,  "topic": user_info["topic"], "target_audience": user_info["target_audience"], "intent": user_info["intent"], "branded_hashtag": user_info["branded_hashtag"], "final_url": user_info["final_url"], "user_input": user_input,  "post_labels_history": []}
    result = llm_chain.run(input_data)
    return result, post_labels_history

def handle_option_12(user_info, post_labels_history):
    scheduler_template = PromptTemplate(template='''  {user_info} {post_labels_history} [Social-AI], You are a world class journalist & Sociall Content Planner.
    Provide the users with an additional bonus.  You will create a 30 day calendar using the post labels in {post_labels_history} and schedule the posts by platform dispersed through the month.  Start at the top with a centered H1 of the {topic}. Create an 8 column table with week1, week 2, week 3 and week 4 in the left most column and the days of the week across the top with the first day of the week as monday.  Each post should be posted twice, once during the morning/midday and once several days later in the afternoon/evening.  Stagger the posting times with no
    post being posted in the midday and then the afternoon for the second post.

  	Starting with the FB posts, distribute them evenly across the month for posting.  Use the labels on the calendar to indicate which
    post occurs on what day. Repeat with linkedin and all of the other posts.  Do not post them all on the same day across sites.
    Stagger them so that there are different posts on different sites every single day of the month.  Use the post labels created at
    the time of post creation to list them on the calendar.

    Current conversation:
    {text}
  	{post_labels_history}
  	Human: {user_input}
  	Social-AI:""   ''', input_variables=["post_labels_history", 'text', 'user_info', "topic", 'user_input',])
    llm_chain = LLMChain(llm=llm, prompt=scheduler_template)
    input_data = {"text": text, "user_info": user_info,  "topic": user_info["topic"],  "user_input": user_input, "post_labels_history": []}
    result = llm_chain.run(input_data)
    return result
#post_labels_history = []  # Initialize post_labels_history as an empty list


In [ ]:

@app.route('/handle_platform', methods=['POST'])
def handle_platform():
    global post_labels_history
    global text, posts, post_summaries
    data = request.get_json()
    print(f"retrieved data for handle_platform {data}")
    selected_platform = int(data.get('platform'))
    print(selected_platform)

    user_info = session.get('user_info')
    if not user_info:
        return jsonify({"error": "No user info set"}), 400  # Return error if user_info is not set
 #   if 'post_labels_history' not in globals():
 #       post_labels_history = []  # Assign a dummy value if #post_labels_history is not defined

    if selected_platform == 1:
        result, post_labels_history = handle_option_1(text, user_info, post_labels_history)
    elif selected_platform == 2:
        print("Handling option 2")
        result, post_labels_history = handle_option_2(posts, user_info, post_labels_history)
    elif selected_platform == 3:
        result, post_labels_history = handle_option_3(post_summaries, user_info, post_labels_history)
    elif selected_platform == 4:
        result, post_labels_history = handle_option_4(post_summaries, user_info, post_labels_history)
    elif selected_platform == 5:
        result, post_labels_history = handle_option_5(posts, user_info, post_labels_history)
    elif selected_platform == 6:
        result, post_labels_history = handle_option_6(posts, user_info, post_labels_history)
    elif selected_platform == 7:
        result, post_labels_history = handle_option_7(posts, user_info, post_labels_history)
    elif selected_platform == 8:
        result, post_labels_history = handle_option_8(post_summaries, user_info, post_labels_history)
    elif selected_platform == 9:
        result, post_labels_history = handle_option_9(text, user_info, post_labels_history)
    elif selected_platform == 10:
        result, post_labels_history = handle_option_10(text, user_info, post_labels_history)
    elif selected_platform == 11:
        result, post_labels_history = handle_option_11(text, user_info, post_labels_history)
    elif selected_platform == 12:
        result, post_labels_history = handle_option_12(user_info, post_labels_history)
    elif selected_platform == 13:
        return jsonify({"message": "Operation terminated for platform 13"})
    else:
        return jsonify({"message": "Invalid platform selected"})

    # Append the results to the chat history
    chat_history.append(result)

    # Return the result and chat history as part of the JSON response
    return jsonify({"content": result, "chat_history": chat_history})

#query = user_input
human_message_prompt = HumanMessagePromptTemplate.from_template(user_input)
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

@app.route('/chat', methods=['POST'])
def chat_route_handler():
    try:
        data = request.get_json()
        print(data)
        message = data.get('message', None)
        print(message)
        conversation_id = None
        print(conversation_id)

        # Create a HumanMessage object with the user input
        if not message or not isinstance(message, str):
            return jsonify({"response": "Invalid question"})

        user_input_message = HumanMessage(content=message)

        # Create a list of messages to pass to the LLM
        messages = [user_input_message]
        print(type(message), message)
        if not message or not isinstance(message, str) or message.isspace():
            return jsonify({"response": "Invalid question"})

        query = {"messages": messages}
        print(query)
        if conversation_id is not None:
            query["chat_history"] = chat_history(conversation_id)

        result = llm(query)
        print(result)
        return jsonify({"response": result["answer"]})

    except Exception as e:
        logging.error(f"An error occurred: {e}")
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    messages = ['message1', 'message2', 'message3']
# Now 'messages' is known to the script, the loop will work
    for i, message in enumerate(messages):
        print(f'{i}: {message}')
    uvicorn.run("app:app", host='0.0.0.0', port=5065, reload=True)


INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:5065 (Press CTRL+C to quit)
INFO:     Started reloader process [156] using WatchFiles


0: message1
1: message2
2: message3
